In [30]:
import numpy as np
from tqdm import tqdm_notebook as tqdm
from tqdm import trange
import torch
import torch.nn as tnn
import torch.nn.functional as F
import torch.optim as topti
from torchtext import data
from torchtext.vocab import GloVe
from imdb_dataloader import IMDB

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device('cpu')

# 测试

In [2]:
textField = data.Field(lower=True, include_lengths=True, batch_first=True)
labelField = data.Field(sequential=False)

from imdb_dataloader import IMDB
train, dev = IMDB.splits(textField, labelField, train="train", validation="dev")

textField.build_vocab(train, dev, vectors=GloVe(name="6B", dim=50))
labelField.build_vocab(train, dev)

trainLoader, testLoader = data.BucketIterator.splits((train, dev), shuffle=True, batch_size=64,
                                                     sort_key=lambda x: len(x.text), sort_within_batch=True)

In [ ]:
lstm1 = tnn.LSTM(50, 200, batch_first=True)

conv = tnn.Conv1d(1, 24, 5, padding=5)
pool = tnn.MaxPool1d(4)
dense = tnn.Linear(200,400)
lstm2 = tnn.LSTM(400, 256,batch_first=True)

In [440]:
conv1 = torch.nn.Conv1d(50, 50, 8, padding=5,).to(device)
pool1 = torch.nn.MaxPool1d(4).to(device)
conv2 = torch.nn.Conv1d(50, 50, 8, padding=5).to(device)
pool2 = torch.nn.MaxPool1d(4).to(device) # shape (batch_size, channel)
conv3 = torch.nn.Conv1d(50, 50, 8, padding=5).to(device)
global_pool = torch.nn.functional.max_pool1d
dense = torch.nn.Linear(1, 1).to(device)

In [450]:
shapes = []
for i, batch in enumerate(trainLoader, 0):
                inputs, length, labels = textField.vocab.vectors[batch.text[0]].to(device), batch.text[1].to(
                device), batch.label.type(torch.FloatTensor).to(device)
                labels -= 1
                o = conv1(inputs.permute(0,2,1))
                o = pool1(o)
                o = conv2(o)
                o = pool2(o)
                o = conv3(o)
                o = global_pool(o,kernel_size=o.shape[2])
                shapes.append(o.shape)


In [451]:
shapes

[torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size([64, 50, 1]),
 torch.Size(

In [436]:
inputs = torch.ones(1,1,50).to(device)

o = conv1(inputs.permute(0,2,1))
o = pool1(o)
o = conv2(o)
o = pool2(o)
o = conv3(o)
o = torch.nn.functional.max_pool1d(o.permute(0,2,1),kernel_size=o.shape[1])
o.shape

torch.Size([1, 4, 1])

In [ ]:
print(o[1][0].permute(1,0,2).shape)

In [ ]:
shapes = []
for i, batch in enumerate(trainLoader, 0):
                inputs, length, labels = textField.vocab.vectors[batch.text[0]].to(device), batch.text[1].to(
                device), batch.label.type(torch.FloatTensor).to(device)
                labels -= 1
                o  = lstm1(inputs)[1][0].permute(1,0,2)   
                #o  = conv(o)
                #o = pool(o)
                o = dense(o.view(-1,200))
                o = lstm2(o.view(-1,1,400))[1][0]
                shapes.append(o.shape)


In [ ]:
o  = lstm(inputs)[1][0].permute(1,0,2)

In [ ]:
o.shape

# 构建神经网络

In [60]:
# Class for creating the neural network.
class Network(tnn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        self.optimizer = None
        self.criterion = None     
        
        self.lstm1 = tnn.LSTM(50, 300, 1,batch_first=True)
        self.relu = tnn.ReLU()
        self.dropout = tnn.Dropout(0.3)
        self.dense1 = tnn.Linear(300, 200)
        self.dense2 = tnn.Linear(200, 128)
        self.dense3 = tnn.Linear(128,64)
        self.dense4 = tnn.Linear(64,1)
        


        
        self.to(self.device)
        print(self.device) 
        
    def forward(self, input, length):
        x = self.lstm1(input)[1][0]
        x = torch.tanh(x)
        x = self.dense1(x)
        x = self.relu(x)
        x = self.dense2(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.dense3(x)
        x = self.dropout(x)
        x = self.dense4(x)

        return x
        
        
    def compile(self, optimizer, criterion):
        self.optimizer = optimizer
        self.criterion = criterion

    def fit(self, trainset, EPOCHS):

        for epoch in range(EPOCHS):
            running_loss = 0.0
            t_iter = tqdm(trainset)
            for i, batch in (enumerate(t_iter, 0)):
                inputs, length, labels = textField.vocab.vectors[batch.text[0]].to(device), batch.text[1].to(
                device), batch.label.type(torch.FloatTensor).to(device)
                labels -= 1
                
                predict = self(inputs.to(self.device),length)
                loss = criterion(predict.view(-1), labels.to(self.device))
                loss.backward()
                optimizer.step()
                
                running_loss += loss.item()
                if i % 32 == 31:
                    t_iter.set_postfix_str("Loss: %.3f" %(running_loss / 32))
                running_loss = 0
        print('trainning completed!')
        
    def save(self, PATH='.model.pth'):
        torch.save(self.state_dict(), PATH)
        print('file saved!')
        
    def evaluate(self, testset):
        correct = 0
        total = 0
        
        with torch.no_grad():
            for i, batch in enumerate(testset, 0):
                inputs, length, labels = textField.vocab.vectors[batch.text[0]].to(device), batch.text[1].to(
                device), batch.label.type(torch.FloatTensor).to(device)
                labels -= 1
                outputs = self(inputs.to(device),length).view(-1)
                predicted = (outputs >= 0.5).to(int) # 用来把simoid后的结果变成非0即1
                total += labels.shape[0]
                correct += (predicted == labels.to(device)).sum().item()
                
        
        print(total)
        print(correct)
        print(f'Accuracy of the network on the {total} test samples: %d %%' % (
            100 * correct / total))
                
            
    def predict(self, batch):
        inputs, length, labels = textField.vocab.vectors[batch.text[0]].to(device), batch.text[1].to(
                device), batch.label.type(torch.FloatTensor).to(device)
        labels -= 1
        with torch.no_grad():
            outputs = self(inputs.to(device),length)
            predicted = (outputs >= 0.5).to(int)
        print('Predition --> \t  ',predicted)
        print('Ground Trueth --> ',labels.to(self.device))
      

In [64]:
net = Network()
print(net)



cuda:0
Network(
  (lstm1): LSTM(50, 300, batch_first=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.3, inplace=False)
  (dense1): Linear(in_features=300, out_features=200, bias=True)
  (dense2): Linear(in_features=200, out_features=128, bias=True)
  (dense3): Linear(in_features=128, out_features=64, bias=True)
  (dense4): Linear(in_features=64, out_features=1, bias=True)
)


In [62]:
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
criterion = tnn.BCEWithLogitsLoss()
net.compile(optimizer, criterion)

In [63]:
net.fit(trainLoader,1)

KeyboardInterrupt: 

In [48]:
net.evaluate(testLoader)

6248
3124
Accuracy of the network on the 6248 test samples: 50 %


In [ ]:
class PreProcessing():
    def pre(x):
        """Called after tokenization"""
        return x

    def post(batch, vocab):
        """Called after numericalization but prior to vectorization"""
        return batch, vocab

    text_field = data.Field(lower=True, include_lengths=True, batch_first=True, preprocessing=pre, postprocessing=post)

In [ ]:
def lossFunc():
    """
    Define a loss function appropriate for the above networks that will
    add a sigmoid to the output and calculate the binary cross-entropy.
    """

In [ ]:
def main():
    # Use a GPU if available, as it should be faster.
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("Using device: " + str(device))

    # Load the training dataset, and create a data loader to generate a batch.
    textField = PreProcessing.text_field
    labelField = data.Field(sequential=False)

    train, dev = IMDB.splits(textField, labelField, train="train", validation="dev")

    textField.build_vocab(train, dev, vectors=GloVe(name="6B", dim=50))
    labelField.build_vocab(train, dev)

    trainLoader, testLoader = data.BucketIterator.splits((train, dev), shuffle=True, batch_size=64,
                                                         sort_key=lambda x: len(x.text), sort_within_batch=True)

    net = Network().to(device)
    criterion =lossFunc()
    optimiser = topti.Adam(net.parameters(), lr=0.001)  # Minimise the loss using the Adam algorithm.

    for epoch in range(10):
        running_loss = 0

        for i, batch in enumerate(trainLoader):
            # Get a batch and potentially send it to GPU memory.
            inputs, length, labels = textField.vocab.vectors[batch.text[0]].to(device), batch.text[1].to(
                device), batch.label.type(torch.FloatTensor).to(device)

            labels -= 1

            # PyTorch calculates gradients by accumulating contributions to them (useful for
            # RNNs).  Hence we must manually set them to zero before calculating them.
            optimiser.zero_grad()

            # Forward pass through the network.
            output = net(inputs, length)

            loss = criterion(output, labels)

            # Calculate gradients.
            loss.backward()

            # Minimise the loss according to the gradient.
            optimiser.step()

            running_loss += loss.item()

            if i % 32 == 31:
                print("Epoch: %2d, Batch: %4d, Loss: %.3f" % (epoch + 1, i + 1, running_loss / 32))
                running_loss = 0

    num_correct = 0

    # Save mode
    torch.save(net.state_dict(), "./model.pth")
    print("Saved model")

    # Evaluate network on the test dataset.  We aren't calculating gradients, so disable autograd to speed up
    # computations and reduce memory usage.
    with torch.no_grad():
        for batch in testLoader:
            # Get a batch and potentially send it to GPU memory.
            inputs, length, labels = textField.vocab.vectors[batch.text[0]].to(device), batch.text[1].to(
                device), batch.label.type(torch.FloatTensor).to(device)

            labels -= 1

            # Get predictions
            outputs = torch.sigmoid(net(inputs, length))
            predicted = torch.round(outputs)

            num_correct += torch.sum(labels == predicted).item()

    accuracy = 100 * num_correct / len(dev)

    print(f"Classification accuracy: {accuracy}")

if __name__ == '__main__':
    main()